<a href="https://colab.research.google.com/github/ixxxxu/deep-learning-from-scratch/blob/master/4_%EC%8B%A0%EA%B2%BD%EB%A7%9D_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 한글폰트 설치방법

In [ ]:
# #한글폰트설치
# !apt-get update -qq
# !apt-get install fonts-nanum* -qq

In [ ]:
# # 폰트매니저 리빌드
# import matplotlib.font_manager as fm
# fm._rebuild()

In [ ]:
# # 한글폰트 설치 확인
# for fontInfo in fm.fontManager.ttflist:
#   if 'Nanum' in fontInfo.name:
#     print(fontInfo.name + " = " + fontInfo.fname)

In [ ]:
# # 한글폰트 설정
# import matplotlib.pyplot as plt
# plt.rc('font', family='NanumGothic') # 나눔고딕으로 한글 폰트 설정

#### MNIST 데이터셋 로드

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(mnist)

#### 예제 실습용 메소드


In [ ]:
# common/functions.py 
  # softmax()
  # cross_entropy_error()

# coding: utf-8
import numpy as np


def identity_function(x):
    return x


def step_function(x):
    return np.array(x > 0, dtype=np.int)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))    


def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)
    

def relu(x):
    return np.maximum(0, x)


def relu_grad(x):
    grad = np.zeros(x)
    grad[x>=0] = 1
    return grad
    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))


def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)
# common/gradient.py
  # numerical_gradient()

import numpy as np

def _numerical_gradient_1d(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        
    return grad


def numerical_gradient_2d(f, X):
    if X.ndim == 1:
        return _numerical_gradient_1d(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_1d(f, x)
        
        return grad


def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

#### 지난시간에


 **학습이란** 
- 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것

**손실함수** 란
- 신경망이 학습할 수 있도록 해 주는 **지표**


**4.1.1 데이터 **주도** 학습**
- 데이터에서 **특징(Feature)**를 추출하고, 그 특징의 패턴을 기계가 학습하는 것 (지도/비지도/강화학습 등)

**4.1.2 훈련데이터와 시험데이터**
- 학습모델의 평가를 위해 훈련데이터와 시험데이터를 분리
  - 오버피팅(Over Fitting 과대적합) : 한 데이터셋에만 지나치게 최적화된 상태  

**4.2 손실함수(Loss Function)**
- 학습모델성능의 '나쁨'을 나타내는 지표 
    - **평균 제곱 오차 MSE**
      - 소프트맥스 함수  

    - **교차 엔트로피 오차 CEE**
      - 로그

**4.2.3 미니배치 학습**
- e.g. 훈련데이터가 100개 == 손실 함수의 값 100개
  - 모든 데이터의 손실함수를 모두 구하는 것은 비효율적
  - 평균손실 함수를 구하여 사용
  - 미니배치 학습
    - 훈련 데이터로부터 일부만 골라 학습을 수행

**4.2.4 교차 엔트로피 오차 구현**

**4.2.5 왜 손실 함수를 설정하는가 ?**
- 높은 '정확도'를 끌어내는 매개변수(가중치와 편향) 값을 찾기 위함
  - 미분을 통하여 손실함수의 값을 가능한 한 작게 하는 매개변수 값을 찾음
  - 반대로 정확도를 지표삼아 미분을 하면 대부분의 장소에서 0이 됨 (신경망 학습에서 계단함수를 쓰지 않는 이유)
  - 신경망학습에서 중요한 성질은 , 기울기가 0이 되지 않게 하는것 (신경망 학습에서 시그모이드 함수를 쓰는 이유)

**4.3 수치 미분**
- 경사법 = 기울기(경사) 값을 기준으로 나아갈 방향을 정함

**4.3.1 미분**
- 한순간의 변화량
- 수치 미분
  - 아주 작은 차분으로 미분 하는 것
  - 반올림 오차 rounding error : 너무 작은 값을 컴퓨터로 계산하는 데 문제가 됨
    - '근사치'로 계산하는 방법
      - 중심차분 / 중앙 차분 : x를 중심으로 그 전후의 차분을 계산
      - 수전방 차분 : (x+h)와 x의 차분
        - 차분 differencing : 연이은 관측값들의 차이를 계산하는 것

- 해석적 미분 : 수식을 전개하여 미분하는 것

**4.3.2 수치 미분의 예**

**4.3.3 편미분**
- 변수가 여럿인 함수에 대한 미분

#### 4.4기울기 


In [ ]:
# coding: utf-8
# cf.http://d.hatena.ne.jp/white_wheels/20100327/p3
import numpy as np
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D


def _numerical_gradient_no_batch(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h 
        fxh2 = f(x) 
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원
        
    return grad


def numerical_gradient(f, X):
    if X.ndim == 1:
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_no_batch(f, x)
        
        return grad

def function_2(x):
    if x.ndim == 1:
        return np.sum(x**2)
    else:
        return np.sum(x**2, axis=1)


def tangent_line(f, x):
    d = numerical_gradient(f, x)
    print(d)
    y = f(x) - d*x
    return lambda t: d*t + y
     

display(numerical_gradient(function_2, np.array([3.0, 4.0])))
display(numerical_gradient(function_2, np.array([0.0, 2.0])))
display(numerical_gradient(function_2, np.array([3.0, 0.0])))

In [ ]:

if __name__ == '__main__':
    x0 = np.arange(-2, 2.5, 0.25)
    x1 = np.arange(-2, 2.5, 0.25)
    X, Y = np.meshgrid(x0, x1)
    
    X = X.flatten()
    Y = Y.flatten()
    
    grad = numerical_gradient(function_2, np.array([X, Y]) )
    
    plt.figure(figsize=(10,10))
    plt.subplot(2,2,1)
    plt.quiver(X, Y, -grad[0], -grad[1],  angles="xy",color="#666666")
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.xlabel('x0')
    plt.ylabel('x1')
    plt.title("<f(x0,x1 = x0 **2 + x1**2>")

    plt.subplot(2,2,2)
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.xlabel('x0')
    plt.ylabel('x1')
    plt.title("<f(x0,x1 = x0 **2 + x1**2>")
    plt.quiver(X, Y, -grad[0], -grad[1],  angles="xy",headwidth=10,scale=40,color = 'r')
    plt.grid()
    # plt.legend()
    plt.draw()
    plt.show();

  - #### 기울기가 가르키는 쪽은 각 장소에서 **함수의 출력값**을 **가장 크게 줄이는 방향**을 타나냄


#### 4.4.1 경사하강법


- 안장점
- 고원
- 경사법
- 경사 하강법 
- 경사 상승법
- **학습률 Learning rate** : 한번의 학습으로 얼마만큼 학습해야 할지(매개변수 값을 얼마나 갱신할지)를 정하는 것
  - 0.01 - 0.001 등 특정 값을 선택
  - 너무 크거나 작으면 '**좋은 장소**'를 찾아 갈 수 없음 
    - **좋은 장소**란 무엇일까요 ?

- **하이퍼파라미터 Hyper Parameter** (초매개변수)
  - 가중치와 편향 같은 신경망의 매개변수와는 성질 이 다른 매개변수
  - 신경망의 가중치 매개변수는 훈련데이터와 학습 알고리즘 에 의해서 '자동'으로 획득됨
  - 학습률 같은 하이퍼파라미터는 **사람이 직접 설정해야 하는 매개변수** 
  



In [ ]:
# coding: utf-8
import numpy as np
import matplotlib.pylab as plt


# 경사하강법 함수 구현
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    x_history = []

    for i in range(step_num):
        x_history.append( x.copy() )

        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x, np.array(x_history)


def function_2(x):
    return x[0]**2 + x[1]**2

# 경사하강법 함수 시각화
init_x = np.array([-3.0, 4.0]) # 초기값 (-3.0 , 4.0) 

lr = 0.1 # Learning rate 학습률
step_num = 20
x, x_history = gradient_descent(function_2, init_x, lr=lr, step_num=step_num)

plt.plot( [-5, 5], [0,0], '--b')
plt.plot( [0,0], [-5, 5], '--b')
plt.plot(x_history[:,0], x_history[:,1], 'o')

plt.xlim(-3.5, 3.5)
plt.ylim(-4.5, 4.5)
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()

In [ ]:
init_x = np.array([-3.0, 4.0])    

lr = 0.1
step_num = 20
gradient_descent(function_2, init_x, lr=lr, step_num=step_num)

#최소값 (0,0)에 가까운 값을 경사법으로 얻은것을 확인 할 수 있다

In [ ]:
# 학습률이 너무 클때
init_x = np.array([-3.0, 4.0])    

lr = 10.0
step_num = 20
gradient_descent(function_2, init_x, lr=lr, step_num=step_num)

# >>> 너무 큰값을 발산

In [ ]:
# 학습률이 너무 작을때
init_x = np.array([-3.0, 4.0])    

lr = 1e-10
step_num = 20
gradient_descent(function_2, init_x, lr=lr, step_num=step_num)

# >>> 거의 갱신되지 않은채 끝남

In [ ]:
# 경사하강법 구현 함수

# coding: utf-8
import numpy as np
import matplotlib.pylab as plt
# from gradient_2d import numerical_gradient

def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    x_history = []

    for i in range(step_num):
        x_history.append( x.copy() )

        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x, np.array(x_history)

In [ ]:
# f(x0,x1) = x0 **2 + x1 **2 구하기

def function_2(x):
  return x

#### 4.4.2 신경망에서의 기울기


In [ ]:
# coding: utf-8
# import sys, os
# sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
# from common.functions import softmax, cross_entropy_error
# from common.gradient import numerical_gradient


class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

net = simpleNet()

f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

print(dW)

- 신경망 학습에서도 기울기를 구해야함
- 가중치 매개변수에 대한 손실 함수의 기울기

In [ ]:
net = simpleNet()
print(net.W) # 가중치 매개변수

In [ ]:
x = np.array([0.6,.09])
p = net.predict(x)
print(p)

In [ ]:
np.argmax(p) # 최댓값의 인덱스

In [ ]:
t = np.array([0,0,1])
net.loss(x,t)

In [ ]:
# 기울기 값 구하기
def f(W):
  return net.loss(x,t)
dW = numerical_gradient(f,net.W)
print(dW)

#### 4.5 학습알고리즘 구현하기


전제
- 신경망에 적응 가능한 가중치와 편향이 있다
- 이 가중치와 편향ㅇ르 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다

신경망 학습 단계 (확률적 경사 하강법 stochastic gradient descent의 예시)
1.   미니 배치
  - 훈련 데이터중 일부를 무작위 선별한 데이터
  - 미니배치의 손실함수 값을 줄이는것을 목표로 함
2.   기울기 산출
  - 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기 산출
  - 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시
3.   매개변수 갱신
- 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
4.   1~3단계 반복

확률적 경사 하강법
- 미니배치를 통한 무작위 확률적 데이터에 대한 수행 방법이라 하여 SGD라고도 함







In [ ]:
print(os.getcwd())

#### 4.5.1 2층 신경망 클래스 구현하기


In [ ]:
#  TwoLayerNet 클래스 

# coding: utf-8
# import sys, os
# sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
# from common.functions import *
# from common.gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

In [ ]:
# coding: utf-8
# import sys, os
# sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
# from two_layer_net import TwoLayerNet

# 데이터 읽기


(x_train, t_train), (x_test, t_test) = mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수를 적절히 설정한다.
train_size = x_train.shape[0]
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

#### 4.5.3 시험데이터로 평가하기


In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수를 적절히 설정한다.
train_size = x_train.shape[0]
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()